In [44]:
from dataset.compas import CompasDataset
from dataset import dataset_loader
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

In [45]:
%reload_ext autoreload
%autoreload 2

In [46]:
name = 'compas'
dataset_ares = dataset_loader(name, data_path='rawdata/', dropped_features=[], n_bins=None)

dataset = CompasDataset(dataset_ares=dataset_ares)
df = dataset.get_dataframe()

In [47]:
df.head()

,Sex = Female,Sex = Male,Age_Cat = Less than 25,Age_Cat = 25 - 45,Age_Cat = Greater than 45,Race = African-American,Race = Asian,Race = Caucasian,Race = Hispanic,Race = Native American,Race = Other,C_Charge_Degree = F,C_Charge_Degree = M,Priors_Count,Time_Served,Status
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10.0,0.0
2,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,14.0,6.0,0.0


### 1.standardization

In [48]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

##### use the standardscaler

In [49]:
# scaler = StandardScaler()

# # Assume df is your DataFrame and 'target' is your target column
# X = df.drop(columns=['Status'])
# Y = df['Status']

# # Step 1: Split the Data
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# X_train_scaler = scaler.fit_transform(X_train)
# X_test_scaler = scaler.transform(X_test)

# print(f'X_train_scaler: {X_train_scaler.shape}')
# print(f'X_test_scaler: {X_test_scaler.shape}')

##### don't use the standardscaler

In [50]:
# Assume df is your DataFrame and 'target' is your target column
X = df.drop(columns=['Status'])
Y = df['Status']

# Step 1: Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train_scaler = X_train.values
X_test_scaler = X_test.values

print(f'X_train_scaler: {X_train_scaler.shape}')
print(f'X_test_scaler: {X_test_scaler.shape}')

X_train_scaler: (4937, 15)
X_test_scaler: (1235, 15)


In [51]:
from models import PyTorchDNN, PyTorchLinearSVM, PyTorchRBFNet,PyTorchLogisticRegression,PyTorchLinearSVM

In [52]:
input_dim = X_train_scaler.shape[1]

model_svm = PyTorchLinearSVM(input_dim)

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()  # 二分类任务中的损失函数
optimizer = optim.Adam(model_svm.parameters(), lr=0.01)  # 使用Adam优化器

# 数据转换为 PyTorch tensor 格式
X_train_tensor = torch.FloatTensor(X_train_scaler)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)  # 确保y是二维



# 训练模型
epochs = 400  # 训练次数
for epoch in range(epochs):
    model_svm.train()  # 进入训练模式
    # 前向传播
    outputs = model_svm(X_train_tensor)
    # 计算损失
    loss = criterion(outputs, y_train_tensor)
    # 梯度清零
    optimizer.zero_grad()
    # 反向传播
    loss.backward()
    # 优化更新参数
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/400], Loss: 1.8101
Epoch [20/400], Loss: 0.8705
Epoch [30/400], Loss: 0.8015
Epoch [40/400], Loss: 0.6938
Epoch [50/400], Loss: 0.6533
Epoch [60/400], Loss: 0.6283
Epoch [70/400], Loss: 0.6186
Epoch [80/400], Loss: 0.6154
Epoch [90/400], Loss: 0.6134
Epoch [100/400], Loss: 0.6118
Epoch [110/400], Loss: 0.6106
Epoch [120/400], Loss: 0.6098
Epoch [130/400], Loss: 0.6092
Epoch [140/400], Loss: 0.6088
Epoch [150/400], Loss: 0.6086
Epoch [160/400], Loss: 0.6084
Epoch [170/400], Loss: 0.6082
Epoch [180/400], Loss: 0.6082
Epoch [190/400], Loss: 0.6081
Epoch [200/400], Loss: 0.6081
Epoch [210/400], Loss: 0.6080
Epoch [220/400], Loss: 0.6080
Epoch [230/400], Loss: 0.6080
Epoch [240/400], Loss: 0.6080
Epoch [250/400], Loss: 0.6080
Epoch [260/400], Loss: 0.6080
Epoch [270/400], Loss: 0.6080
Epoch [280/400], Loss: 0.6080
Epoch [290/400], Loss: 0.6080
Epoch [300/400], Loss: 0.6080
Epoch [310/400], Loss: 0.6080
Epoch [320/400], Loss: 0.6080
Epoch [330/400], Loss: 0.6080
Epoch [340/400], Lo

### 2. initialize MODEL and DATA

In [53]:
from xai_cola.ml_model import Model
from xai_cola import data

In [54]:
column_names = X.columns.tolist()
column_names

['Sex = Female',
 'Sex = Male',
 'Age_Cat = Less than 25',
 'Age_Cat = 25 - 45',
 'Age_Cat = Greater than 45',
 'Race = African-American',
 'Race = Asian',
 'Race = Caucasian',
 'Race = Hispanic',
 'Race = Native American',
 'Race = Other',
 'C_Charge_Degree = F',
 'C_Charge_Degree = M',
 'Priors_Count',
 'Time_Served']

In [55]:
ml_model = Model(model_svm,'PYT')
compas_ares = data.NumpyData(
    data=X_test_scaler, 
    target_name="Status", 
    feature_names=column_names,
    )


##### Test counterfactual_explainer/ares.py

In [56]:
from xai_cola.counterfactual_explainer.ares import ARecourseS
ares_test = ARecourseS(
    ml_model=ml_model,
    data=compas_ares,
    sample_num=2,
    input_factual_num=200,
    dataset_ares=dataset_ares,   #看这部分怎么加到我的dataset 里面去
)

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


Computing Candidate Sets of Conjunctions of Predicates SD and RL
Processing 60 combinations | Sampling itemset size 65
SD and RL Computed with Lengths 123 and 123
Computing Ground Set of Triples V
Reducing RL
RL Reduced from Size 123 to 120


100%|██████████| 123/123 [00:00<00:00, 5141.10it/s]


Ground Set Computed with Length 1950


0it [00:00, ?it/s]np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
Mean of empty slice.
invalid value encountered in scalar divide
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
Mean of empty slice.
invalid value encountered in scalar divide
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
Mean of empty slice.
invalid value encountered in scalar divide
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (D

...

  0%|          | 0/199 [00:00<?, ?it/s]Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
M

Checking Exchange


  0%|          | 0/199 [00:00<?, ?it/s]Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
Mean of empty slice.
invalid value encountered in scalar divide
M

Final sample number is only 0, cannot generate the requested 2 counterfactuals.



Mean of empty slice.


In [57]:
a = ares_test.factual
b = ares_test.counterfactual
a

array([], shape=(0, 15), dtype=float64)

In [58]:
b

array([], shape=(0, 15), dtype=float64)

##### Test the DisCount

In [59]:
from xai_cola.counterfactual_explainer import DisCount

discount_test = DisCount(
    ml_model = ml_model,
    data = compas_ares,
    sample_num = 4,
)

INFO:root:Optimization started
  0%|          | 0/15 [00:00<?, ?it/s]INFO:root:U_1-Qu_upper=0.3860608390944272, U_2-Qv_upper=-0.8802248709794633
INFO:root:eta=1, l=0.24000000000000002, r=1
INFO:root:Iter 1: Q = 9682.1171875, term1 = 132.24896240234375, term2 = 9682.1171875
  7%|▋         | 1/15 [00:00<00:13,  1.04it/s]INFO:root:U_1-Qu_upper=-17.863871797272097, U_2-Qv_upper=-101.72558129352291
INFO:root:eta=0.8864737465132551, l=0.278, r=1
INFO:root:Iter 2: Q = 43755144.0, term1 = 1228392.125, term2 = 49201328.0
 13%|█▎        | 2/15 [00:01<00:12,  1.02it/s]INFO:root:U_1-Qu_upper=-50.09284275026167, U_2-Qv_upper=-27.09785328272916
INFO:root:eta=0.5314586559728474, l=0.278, r=0.9639
INFO:root:Iter 3: Q = 5797160.5, term1 = 1140854.375, term2 = 9902224.0
 20%|██        | 3/15 [00:02<00:11,  1.02it/s]INFO:root:U_1-Qu_upper=-317.8195899122659, U_2-Qv_upper=-381.2180410216534
INFO:root:eta=0.652053474041757, l=0.31229500000000004, r=0.9639
INFO:root:Iter 4: Q = 387485664.0, term1 = 44504204

In [60]:
a,b = discount_test.generate_counterfactuals()

INFO:root:Optimization started
  0%|          | 0/15 [00:00<?, ?it/s]INFO:root:U_1-Qu_upper=0.38792435176642315, U_2-Qv_upper=-0.8179349572961119
INFO:root:eta=1, l=0.24000000000000002, r=1
INFO:root:Iter 1: Q = 12127.9365234375, term1 = 205.7550048828125, term2 = 12127.9365234375
  7%|▋         | 1/15 [00:01<00:14,  1.07s/it]INFO:root:U_1-Qu_upper=-18.41656116893017, U_2-Qv_upper=-110.21172254703345
INFO:root:eta=0.8911857790212444, l=0.278, r=1
INFO:root:Iter 2: Q = 52906388.0, term1 = 1960750.375, term2 = 59126880.0
 13%|█▎        | 2/15 [00:02<00:13,  1.03s/it]INFO:root:U_1-Qu_upper=-25.834896441186743, U_2-Qv_upper=-88.41829043998622
INFO:root:eta=0.836741575970775, l=0.31410000000000005, r=1
INFO:root:Iter 3: Q = 38822668.0, term1 = 1733093.0, term2 = 46059292.0
 20%|██        | 3/15 [00:03<00:12,  1.04s/it]INFO:root:U_1-Qu_upper=-1629.9444266940477, U_2-Qv_upper=-8771.498447296883
INFO:root:eta=0.8925169425229473, l=0.34839500000000007, r=1
INFO:root:Iter 4: Q = 337006297088.0, 